In [85]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import spacy

### Reading our editing Amazon Dataset.

In [86]:
amazon_ds = pd.read_csv("A_amazon_catelogue.csv", encoding = 'Latin-1')
amazon_ds.head()

,ID,ASIN,item_name
0,NaN,B09TLGFJCV,A & D - Strip of 10 Capsules
1,NaN,B0BB6DG4QD,A Bec L - Bottle of 30 Tablets
2,NaN,B0BB6KKQDS,A BEC TABLET
3,NaN,B08BXDM8KL,A Doxid - Strip of 10 Tablets
4,NaN,B0947JD4BX,A Doxid L - Strip of 10 Capsules


### Reading our product master Dataset.

In [87]:
pd_master = pd.read_csv("A_product_master_2.csv")
pd_master.head()

,Product_ID,Client_ID,Product_Code,Product_Desc,UOM,Packsize,Manufacturer,MRP,PTS,PTR,...,Category,molecular_id,P_ID,GST_PER,Manufacturer_Code,ED_MDM_Product_Code,ED_MDM_Product_Image,ED_MDM_Product_Document_Image,ED_MDM_Product_Creation_Time,Match
0,1697255,756,J0002111,A MAX CAPSULES 10^S,NaN,10'S,DERMAWIZ LAB DERMA,155.0,108.50,108.50,...,NaN,NaN,117107987.0,NaN,1h,ED000157592,NaN,NaN,NaN,NaN
1,1696011,756,J0002112,A PLUS TABLETS,NaN,NaN,MEDLIFE GENETICS-LAM,230.0,0.00,155.90,...,NaN,NaN,NaN,NaN,GXN,ED000148859,NaN,NaN,NaN,NaN
2,1714647,756,J0002113,A TO Z GOLD CAPSULES NS 15CAPSULES,NaN,15CAP,ALKEM #M CARE#,210.0,142.37,142.39,...,NaN,NaN,117111597.0,NaN,BB,ED000000174,NaN,NaN,NaN,NaN
3,1686756,756,J0002115,A TO Z GOLD CAPSULES 15^S,NaN,15'S,ALKEM ALTIS,210.0,128.14,142.37,...,NaN,NaN,117124974.0,NaN,GXQ,ED000155268,NaN,NaN,NaN,NaN
4,7790857,756,A00849,A TO Z IMMUNE TABLETS 1*16,NaN,1*16,ALKEM #M CARE#,215.0,145.76,145.76,...,NaN,NaN,118559972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
amazon_ds['item_name']

0                            A & D - Strip of 10 Capsules
1                          A Bec L - Bottle of 30 Tablets
2                                            A BEC TABLET
3                           A Doxid - Strip of 10 Tablets
4                        A Doxid L - Strip of 10 Capsules
                              ...                        
4280                    Azuvas 40MG - Strip of 10 Tablets
4281                   Azysafe 500Mg - Strip of 5 Tablets
4282                     Azysafe-500 - Strip of 5 Tablets
4283                   AZYTIME 500MG - Strip of 3 Tablets
4284    Azzura Essential Renal Nutrition Vanilla Flavo...
Name: item_name, Length: 4285, dtype: object

## In amazon Dataframe.

### Spliting the "item_name" column along "-".

In [89]:
for items1 in amazon_ds['item_name']:
    string = str(items1)
    last_dash_index = string.rfind("-")
    
    if last_dash_index != -1:
        first_part = string[:last_dash_index]
        second_part = string[last_dash_index + 1:]
        dict1 = {first_part: second_part}
    else:
        string1 = string.replace(" ", "")
#         print(string1)
    keys = list(dict1.keys())  # Convert dict_keys object to a list
    
    for i in keys:
        i = i.replace(" ", "")
#         print(i)
    

### Assigning two columns for the medicine name and the type.

In [90]:
df = amazon_ds[['item_name']] 
df[['First_Part', 'Type']] = df['item_name'].str.rsplit('-', n=1, expand=True)

df['First_Part'] = df['First_Part'].str.replace(" ", "")
df['First_Part'] = df['First_Part'].str.upper()

df.head()

C:\Users\mohammad\AppData\Local\Temp\ipykernel_34968\1495230820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['First_Part', 'Type']] = df['item_name'].str.rsplit('-', n=1, expand=True)


,item_name,First_Part,Type
0,A & D - Strip of 10 Capsules,A&D,Strip of 10 Capsules
1,A Bec L - Bottle of 30 Tablets,ABECL,Bottle of 30 Tablets
2,A BEC TABLET,ABECTABLET,None
3,A Doxid - Strip of 10 Tablets,ADOXID,Strip of 10 Tablets
4,A Doxid L - Strip of 10 Capsules,ADOXIDL,Strip of 10 Capsules


In [91]:
df.shape

(4285, 3)

## In Product Master Dataframe.

### 

In [92]:
df_2 = pd.read_csv("File.csv")

In [93]:
df_2.head()

,Unnamed: 0,Product_Desc,First_part,Type
0,0,A MAX CAPSULES 10^S,AMAX,CAPSULES Strip of 10
1,1,A PLUS TABLETS,APLUS,TABLETS
2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES
3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,CAPSULES Strip of 15
4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,TABLETS 1*16


In [94]:
df_2.shape

(3449, 4)

In [95]:
for x in df["First_Part"]:
    for y in df_2["First_part"]:
        similarity_score = fuzz.ratio(x,y)
        if similarity_score >= 85:
            print(x, "------", y, similarity_score)

ASTAR-P ------ ASTARP 92
ASTAR-P ------ ASTARP 92
ATOZGOLD ------ ATOZGOLD 100
ATOZGOLD ------ ATOZGOLD 100
ATOZIMMUNE ------ ATOZIMMUNE 100
ATOZIMMUNE ------ ATOZIMMUNE 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS+ 92
ATOZNS ------ ATOZNS 100
ATOZNS ------ ATOZNS 100
ATOZWOMEN ------ ATOZWOMAN 89
ATRET10MG ------ ACUTRET10MG 90
ATRET10MG ------ ACUTRET10MG 90
ATRET10MG ------ ACUTRET10MG 90
AACTARIL ------ AACTARIL 

ACIVIR-200DT ------ ACIVIR200DT 96
ACIVIR-200DT ------ ACIVIR200DT 96
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR400DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-200DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR200DT 87
ACIVIR-400DT ------ ACIVIR200DT 87
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR400DT 96
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-400DT ------ ACIVIR800DT 87
ACIVIR-800DT ------ ACIVIR200DT 87
ACIVIR-800DT ------ ACIVIR200DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR400DT 87
ACIVIR-800DT ------ ACIVIR800DT 96
ACIVIR-800DT ------ ACIVIR800DT 96
ACIVIR-800DT ------ ACIVIR800DT 96
ACLAIR ------ ALOCLAIR 86
ACLINDBP2.5% ------ ACLINDBP2.5 96
ACLINDBP2.5% ------ ACLINDBP5% 91
ACLINDBP5% ------ ACLINDBP2.5 86
ACLINDBP5% ------ ACLINDBP5% 100

AEROMONT-B ------ AEROMONTB 95
AEROMONT-B ------ AEROMONTB 95
AEROTROP ------ AEROTROP 100
AEROTROP ------ AEROTROPF 94
AEROTROPF ------ AEROTROP 94
AEROTROPF ------ AEROTROPF 100
AEROTROP-F ------ AEROTROP 89
AEROTROP-F ------ AEROTROPF 95
AF-C ------ AF-C 100
AFDERM-MN ------ AFDERMMN 94
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN 94
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDERM-MN ------ AFDERMMN+ 89
AFDURA ------ AFDURA 100
AFENACP ------ ACENACP 86
AFENACP ------ ACENACP 86
AFENACP ------ AFENACP 100
AFENACP ------ ALFENAC 86
AFITRA-200 ------ AFITRA200 95
AF-KIT ------ AF-KIT4 92
AFLAB2 ------ AFLAB2 100
AFOGATRAN110 ------ AFOGATRAN110 100
AFOGATRAN110 ------ AFOGATRAN150 92
AFOGATRAN150 ------ AFOGATRAN110 92
AFOGATRAN150 ------ AFOGATRAN150 100
AFOGATRAN150 ------ AFOGATRAN75 87
AFOGATRAN75 ------ AFOGATRAN150 87
AFOGATRAN75 ------ AFOGATRAN75 100
AFOGLIP ------ AFOGLIP 100
AFOGLIP ---

ALFAGABA100MG ------ ALFAGABA100 92
ALFA-MAXICAL ------ ALFA-MAXICAL 100
ALFATAM-D ------ ALFATAMD 94
ALFENCE ------ ALFENAC 86
ALFOO10MG ------ ALFOO10 88
ALFOO10MG ------ ALFOO10 88
ALFUGRESS10MG ------ ALFUGRESS1*10 85
ALFUSIND ------ ALFUSIND# 94
ALFUSIND ------ ALFUSIND 100
ALFUSIND ------ ALFUSIN 93
ALFUSIND ------ ALFUSIN 93
ALGEST200 ------ ALGEST200 100
ALGEST200 ------ ALGESTSR200 90
ALGESTSR200 ------ ALGEST200 90
ALGESTSR200 ------ ALGESTSR200 100
ALGESTSR200 ------ ALGESTSR300 91
ALGESTSR300MG ------ ALGESTSR300 92
ALGOSSPAS ------ ALGOSSPAS 100
ALIVHER ------ ALIVHERES 88
ALIVHER ------ ALIVHER 100
ALIVHER ------ ALIVHER 100
ALIVHER ------ ALIVHER 100
ALIVHER ------ ALIVHER 100
ALIZAP ------ ALIZAP 100
ALKA5 ------ ALKA-5 91
ALKACIP ------ ALKACIP 100
ALKACIP ------ ALKACIP 100
ALKACITRAL ------ ALKACITRAL 100
ALKASHOTB6S/F ------ ALKASHOTB6 87
ALKASTON ------ ALKASTONB6 89
ALKASTON ------ ALKASTON 100
ALKASTON ------ ALKASTON 100
ALKASTON ------ ALKASTONB6 89
ALKASTON --

AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODILXP 90
AMBRODILLX ------ AMBRODILXP 90
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODILLX 100
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODIL 89
AMBRODILLX ------ AMBRODILXP 90
AMBRODILLX ------ AMBRODILXP 90
AMBRODILXP ------ AMBRODIL 89
AMBRODILXP ------ AMBRODILLX 90
AMBRODILXP ------ AMBRODILLX 90
AMBRODILXP ------ AMBRODILLX 90
AMBRODILXP ------ AMBRODIL 89
AMBRODILXP ------ AMBRODIL 89
AMBRODILXP ------ AMBRODIL 89
AMBRODILXP ------ AMBRODILXP 100
AMBRODILXP ------ AMBRODILXP 100
AMBRODIL-D ------ AMBRODILD 95
AMBRODIL-D ------ AMBRODILD 95
AMBRODIL-D ------ AMBRODIL 89
AMBRODIL-D ------ AMBRODIL 89
AMBRODIL-D ------ AMBRODIL 89


...

ARKAMIN ------ ARKAMIN 100
ARKAMIN ------ ARKAMIN 100
ARKAMIN ------ ARKAMIN 100
ARKAMIN ------ ARKAMIN-H 88
ARKAMINH ------ ARKAMIN 93
ARKAMINH ------ ARKAMIN 93
ARKAMINH ------ ARKAMIN 93
ARKAMINH ------ ARKAMIN-H 94
ARK-AP ------ ARKAP 91
AROFF-EZ ------ AROFFEZ 93
AROLAC ------ AMROLMAC 86
AROLAC ------ AMROLMAC 86
AROPHYLL-HD ------ AROPHYLLHD 95
ARPIMUNEME100MG ------ ARPIMUNEME100MG5.S 91
ARPIZOL10 ------ ARPIZOL210S 90
ARPIZOL20 ------ ARPIZOL210S 90
ARRENO ------ ARRENO 100
ARTIFLO1MG ------ ARTIFLO1 89
ARTIFLO1MG ------ ARTIFLO1 89
ARTIFLO-2 ------ ARTIFLO2 94
ARTIFLO-3 ------ ARTIFLO3 94
ARVAST10 ------ ARVAST10 100
ARVAST10 ------ ARVAST10 100
ARVAST10 ------ ARVAST15 88
ARVAST10 ------ ARVAST20 88
ARVAST10 ------ ARVAST30 88
ARVAST10 ------ ARVAST40 88
ARVAST10 ------ ARVASTA150 89
ARVAST10 ------ ARVASTCF10 89
ARVAST10 ------ ARVASTCV10 89
ARVAST10 ------ ARVASTF10 94
ARVAST10 ------ AVAS10 86
ARVAST10 ------ AVAS10 86
ARVAST10 ------ AVAS10 86
ARVAST15 ------ ARVAST10 88

ASTHALINROTACAPS ------ ASTHALINROTA 86
ASTHALINROTACAPS ------ ASTHALINROTA 86
ASTHALINROTACAPS ------ ASTHALINROTA 86
ASTHALINROTACAPS ------ ASTHALINROTA 86
ASTHALIN-2 ------ ASTHALIN2 95
ASTHALIN-2 ------ ASTHALIN2 95
ASTHALIN-2 ------ ASTHALIN 89
ASTHALIN-2 ------ ASTHALIN 89
ASTHALIN-2 ------ ASTHALIN2 95
ASTHALIN-2 ------ ASTHALIN2 95
ASTHALIN-2 ------ ASTHALIN 89
ASTHALIN-2 ------ ASTHALIN 89
ASTHALIN-4 ------ ASTHALIN4 95
ASTHALIN-4 ------ ASTHALIN4 95
ASTHALIN-4 ------ ASTHALIN 89
ASTHALIN-4 ------ ASTHALIN 89
ASTIN10MG ------ ASTIN10 88
ASTIN10MG ------ ASTIN10 88
ASTIN10MG ------ ASTIN10 88
ASTIN20MG ------ ASTIN20 88
ASTIN20MG ------ ASTIN20 88
ASTIN20MG ------ ASTIN20 88
ASTIN20MG ------ ASTIN20 88
ASTINCV10 ------ ASTIN10 88
ASTINCV10 ------ ASTIN10 88
ASTINCV10 ------ ASTIN10 88
ASTINCV10 ------ ASTINCV10 100
ASTINCV10 ------ ASTINCV10 100
ASTINCV10 ------ ASTINCV20 89
ASTINCV20 ------ ASTIN20 88
ASTINCV20 ------ ASTIN20 88
ASTINCV20 ------ ASTIN20 88
ASTINCV20 ------ A

ATORINF ------ ATORIN5 86
ATORINF ------ ATORINF 100
ATORIN-10 ------ ATORIN10 94
ATORIN-5 ------ ATORIN5 93
ATORLIPASP-10 ------ ATORLIPASP10 96
ATORLIPASP-10 ------ ATORLIPASP20 88
ATORLIPASP-20 ------ ATORLIPASP10 88
ATORLIPASP-20 ------ ATORLIPASP20 96
ATORLIPCV ------ ATORLIPCV 100
ATORLIPCV-20 ------ ATORLIP20 86
ATORLIPCV-20 ------ ATORLIP20 86
ATORLIPCV-20 ------ ATORLIPCV 86
ATORLIP-F ------ ATORLIPF 94
ATORLIP-F ------ ATORLIPF 94
ATORLIP-10 ------ ATORLIP10 95
ATORLIP-10 ------ ATORLIP10 95
ATORLIP-10 ------ ATORLIP10 95
ATORLIP-10 ------ ATORLIPF10 90
ATORLIP-10 ------ ATORZIP-10 90
ATORLIP-20 ------ ATORLIP20 95
ATORLIP-20 ------ ATORLIP20 95
ATORLIP-40 ------ ATORLIP40 95
ATORLIP-40 ------ ATORLIP40 95
ATORLIP-5 ------ ATORLIP5 94
ATORLIP-5 ------ ATORLIP5 94
ATORLIP-80 ------ ATORLIP80 95
ATORMAC10 ------ ATORMAC10 100
ATORMAC10 ------ ATORMAC20 89
ATORMAC10 ------ ATORMAC40 89
ATORMAC20 ------ ATORMAC10 89
ATORMAC20 ------ ATORMAC20 100
ATORMAC20 ------ ATORMAC40 89
ATO

AZERVA10 ------ AZERVA10 100
AZERVA10 ------ AZERVA20 88
AZERVA10 ------ AZERVA40 88
AZERVA20 ------ AZERVA10 88
AZERVA20 ------ AZERVA20 100
AZERVA20 ------ AZERVA40 88
AZERVA40 ------ AZERVA10 88
AZERVA40 ------ AZERVA20 88
AZERVA40 ------ AZERVA40 100
AZERVAE ------ AZERVAE 100
AZIBACT250MG ------ AZIBACT250 91
AZIBACT250MG ------ AZIBACT250 91
AZIBACT250MG ------ AZIBACT250 91
AZIBACT250MG ------ AZIBACT250 91
AZIBACT500MG ------ AZIBACT500 91
AZIBACTLRREADYMIX100MG ------ AZIBACTREADYMIX10015ML 86
AZIBACTLRREADYMIX200 ------ AZIBACTREADYMIX20015ML 86
AZIBACTREADYMIX100 ------ AZIBACTREADYMIX10015ML 90
AZIBACTREADYMIX100 ------ AZIBACTREADYMIX20015ML 85
AZIBACTREADYMIX200 ------ AZIBACTREADYMIX10015ML 85
AZIBACTREADYMIX200 ------ AZIBACTREADYMIX20015ML 90
AZIBACT-500 ------ AZIBACT250 86
AZIBACT-500 ------ AZIBACT250 86
AZIBACT-500 ------ AZIBACT500SI 87
AZIBACT-500 ------ AZIBACT500 95
AZIBEST200MG ------ AZIBEST200 91
AZIBEST250 ------ AZIBEST200 90
AZIBEST250 ------ AZIBEST250 1

AZORAN ------ AZORAN25 86
AZORAN ------ AZORAN50 86
AZORAN ------ AZORAN50 86
AZORAN25 ------ AZORAN2525 89
AZORAN25 ------ AZORAN25 100
AZORAN25 ------ AZORAN50 88
AZORAN25 ------ AZORAN50 88
AZORAN50MG ------ AZORAN50 89
AZORAN50MG ------ AZORAN50 89
AZOVAS16MG ------ AZOVAS16 89
AZOVIT ------ AZOVIT 100
AZOVIT ------ AZOVITCT 86
AZOVIT ------ AZOVIT 100
AZOVIT ------ AZOVITL 92
AZOVIT ------ AZOVITL 92
AZOVIT ------ AZOVIT 100
AZOVITL ------ AZOVIT 92
AZOVITL ------ AZOVIT 92
AZOVITL ------ AZOVITL 100
AZOVITL ------ AZOVITL 100
AZOVITL ------ AZOVIT 92
AZOVITL ------ AZOVIT 92
AZOVITL ------ AZOVIT 92
AZOVITL ------ AZOVITL 100
AZOVITL ------ AZOVITL 100
AZOVITL ------ AZOVIT 92
AZR ------ AZR 100
AZR ------ AZR 100
AZRO200 ------ AZRO250 86
AZRO250 ------ AZIPRO250 88
AZRO250 ------ AZRO250 100
AZRO500 ------ AZIPRO500 88
AZRO500 ------ AZRO250 86
AZTOLET10 ------ AZTOLET2010S 86
AZTOLET20 ------ AZTOLET2010S 86
AZTOR10 ------ AZTOR10 100
AZTOR10 ------ AZTOR10 100
AZTOR10 ------ 

In [96]:
df.head(20)

,item_name,First_Part,Type
0,A & D - Strip of 10 Capsules,A&D,Strip of 10 Capsules
1,A Bec L - Bottle of 30 Tablets,ABECL,Bottle of 30 Tablets
2,A BEC TABLET,ABECTABLET,None
3,A Doxid - Strip of 10 Tablets,ADOXID,Strip of 10 Tablets
4,A Doxid L - Strip of 10 Capsules,ADOXIDL,Strip of 10 Capsules
5,A K T FD - Strip of 3 Tablets,AKTFD,Strip of 3 Tablets
6,A K T FD - Strip of 3 Tablets,AKTFD,Strip of 3 Tablets
7,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
8,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
9,A To Z - Bottle of 100ml Syrup,ATOZ,Bottle of 100ml Syrup


In [97]:
index_1 = []
index_2 = []
for i, x in enumerate(df["First_Part"]):
    for j, y in enumerate(df_2["First_part"]):
        similarity_score = fuzz.ratio(x, y)
        if similarity_score >= 90:
            index_1.append(i)
            index_2.append(j)

In [98]:
new_df_1 = df.loc[index_1]
new_df_1.head(15)


,item_name,First_Part,Type
7,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
8,A Star-P - Strip of 10 Tablets,ASTAR-P,Strip of 10 Tablets
11,A To Z Gold - Strip of 15 Capsules,ATOZGOLD,Strip of 15 Capsules
11,A To Z Gold - Strip of 15 Capsules,ATOZGOLD,Strip of 15 Capsules
12,A To Z Immune - Strip of 16 Tablets,ATOZIMMUNE,Strip of 16 Tablets
12,A To Z Immune - Strip of 16 Tablets,ATOZIMMUNE,Strip of 16 Tablets
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup
13,A To Z NS - Bottle of 200Ml Syrup,ATOZNS,Bottle of 200Ml Syrup


In [99]:
new_df_1.to_csv("csv1.csv")
new_df_2.to_csv("csv2.csv")

In [102]:
new_df_3 = pd.read_csv("csv2_edited.csv")

In [103]:
new_df_3["Type"] = new_df_3["Type"].str.upper()
new_df_3["Type_2"] = new_df_3["Type_2"].str.upper()

In [104]:
new_df_3.head(15)

,Unnamed: 0.1,Unnamed: 0,Product_Desc,First_part,Type,Type_2
0,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
1,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
2,2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES,STRIP OF 15 CAPSULES
3,3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,STRIP OF 15 CAPSULES,STRIP OF 15 CAPSULES
4,4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,STRIP OF 16 TABLETS,STRIP OF 16 TABLETS
5,5,5,A TO Z IMMUNE TABLETS 16^S,ATOZIMMUNE,TABLETS STRIP OF 16,STRIP OF 16 TABLETS
6,6,6,A TO Z NS + DROP # 15ML,ATOZNS+,DROP # 15ML,BOTTLE OF 200ML SYRUP
7,7,7,A TO Z NS + DROP 5% 15ML,ATOZNS+,DROP 5% 15ML,BOTTLE OF 200ML SYRUP
8,8,8,A TO Z NS + DROPS 15ML,ATOZNS+,DROPS 15ML,BOTTLE OF 200ML SYRUP
9,9,9,A TO Z NS + TABLETS 1*15,ATOZNS+,TABLETS 1*15,BOTTLE OF 200ML SYRUP


In [105]:
new_df_3.to_csv("Final_prolly.csv")

In [110]:
new_df_4 = pd.read_csv("Final_prolly_2.csv",sep = "\t")
new_df_4.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Product_Desc,First_part,Type,Type_2
0,0,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
1,1,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
2,2,2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES,STRIP OF 15 CAPSULES
3,3,3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,STRIP OF 15 CAPSULES,STRIP OF 15 CAPSULES
4,4,4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,STRIP OF 16 TABLETS,STRIP OF 16 TABLETS


In [111]:
matching_indices = []
for index, row in new_df_4.iterrows():
    column1_value = str(row['Type'])  # Convert to string
    column2_value = str(row['Type_2'])  # Convert to string
    
    ratio = fuzz.ratio(column1_value, column2_value)
    if ratio >= 50:
        matching_indices.append(index)
        print(column1_value, '--------', column2_value,"==========", ratio)

STRIP OF 10 TABLETS --------  STRIP OF 10 TABLETS ========== 97
STRIP OF 10 TABLETS --------  STRIP OF 10 TABLETS ========== 97
CAPSULES --------  STRIP OF 15 CAPSULES ========== 55
STRIP OF 15 CAPSULES --------  STRIP OF 15 CAPSULES ========== 98
STRIP OF 16 TABLETS --------  STRIP OF 16 TABLETS ========== 97
TABLETS STRIP OF 16 --------  STRIP OF 16 TABLETS ========== 62
STRIP OF 15 TABLETS --------  STRIP OF 15 TABLETS ========== 97
STRIP OF 15 TABLETS --------  STRIP OF 15 TABLETS ========== 97
CAPSULES --------  STRIP OF 10 CAPSULES ========== 55
STRIP OF 10 CAPSULES --------  STRIP OF 10 CAPSULES ========== 98
CAPSULES(I) STRIP OF 10 --------  STRIP OF 10 CAPSULES ========== 55
SYRUP 100ML --------  100 ML SYRUP ========== 50
STRIP OF 10 TABLETS --------  STRIP OF 10 CAPSULES ========== 80
STRIP OF 10 TABLETS --------  STRIP OF 10 CAPSULES ========== 80
STRIP OF 10 TABLETS --------  STRIP OF 10 TABLETS ========== 97
STRIP OF 10 TABLETS --------  STRIP OF 10 TABLETS ========== 97


In [113]:
new_df_4 = new_df_4.loc[matching_indices]

In [132]:
len(matching_indices)

1269

In [117]:
new_df_4.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Product_Desc,First_part,Type,Type_2
0,0,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
1,1,2601,2601,ASTAR P TABLETS 1*10,ASTARP,STRIP OF 10 TABLETS,STRIP OF 10 TABLETS
2,2,2,2,A TO Z GOLD CAPSULES NS 15CAPSULES,ATOZGOLD,CAPSULES,STRIP OF 15 CAPSULES
3,3,3,3,A TO Z GOLD CAPSULES 15^S,ATOZGOLD,STRIP OF 15 CAPSULES,STRIP OF 15 CAPSULES
4,4,4,4,A TO Z IMMUNE TABLETS 1*16,ATOZIMMUNE,STRIP OF 16 TABLETS,STRIP OF 16 TABLETS


In [125]:
df_jaggi = pd.read_csv("A_product_master_2.csv")

In [133]:
df_jaggi = df_jaggi.loc[matching_indices]

In [135]:
df_jaggi.head()

,Product_ID,Client_ID,Product_Code,Product_Desc,UOM,Packsize,Manufacturer,MRP,PTS,PTR,...,Category,molecular_id,P_ID,GST_PER,Manufacturer_Code,ED_MDM_Product_Code,ED_MDM_Product_Image,ED_MDM_Product_Document_Image,ED_MDM_Product_Creation_Time,Match
0,1697255,756,J0002111,A MAX CAPSULES 10^S,NaN,10'S,DERMAWIZ LAB DERMA,155.0,108.50,108.50,...,NaN,NaN,117107987.0,NaN,1h,ED000157592,NaN,NaN,NaN,NaN
1,1696011,756,J0002112,A PLUS TABLETS,NaN,NaN,MEDLIFE GENETICS-LAM,230.0,0.00,155.90,...,NaN,NaN,NaN,NaN,GXN,ED000148859,NaN,NaN,NaN,NaN
2,1714647,756,J0002113,A TO Z GOLD CAPSULES NS 15CAPSULES,NaN,15CAP,ALKEM #M CARE#,210.0,142.37,142.39,...,NaN,NaN,117111597.0,NaN,BB,ED000000174,NaN,NaN,NaN,NaN
3,1686756,756,J0002115,A TO Z GOLD CAPSULES 15^S,NaN,15'S,ALKEM ALTIS,210.0,128.14,142.37,...,NaN,NaN,117124974.0,NaN,GXQ,ED000155268,NaN,NaN,NaN,NaN
4,7790857,756,A00849,A TO Z IMMUNE TABLETS 1*16,NaN,1*16,ALKEM #M CARE#,215.0,145.76,145.76,...,NaN,NaN,118559972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
a22 = df_jaggi["ED_MDM_Product_Code"]

In [141]:
amazon_ds.head()

,ID,ASIN,item_name
0,NaN,B09TLGFJCV,A & D - Strip of 10 Capsules
1,NaN,B0BB6DG4QD,A Bec L - Bottle of 30 Tablets
2,NaN,B0BB6KKQDS,A BEC TABLET
3,NaN,B08BXDM8KL,A Doxid - Strip of 10 Tablets
4,NaN,B0947JD4BX,A Doxid L - Strip of 10 Capsules


In [147]:
amazon_ds = amazon_ds.join(a22)

In [148]:
amazon_ds.head()

,ID,ASIN,item_name,ED_MDM_Product_Code
0,NaN,B09TLGFJCV,A & D - Strip of 10 Capsules,ED000157592
1,NaN,B0BB6DG4QD,A Bec L - Bottle of 30 Tablets,ED000148859
2,NaN,B0BB6KKQDS,A BEC TABLET,ED000000174
3,NaN,B08BXDM8KL,A Doxid - Strip of 10 Tablets,ED000155268
4,NaN,B0947JD4BX,A Doxid L - Strip of 10 Capsules,NaN


In [149]:
amazon_ds.to_csv("amazon_catalogue_modified.csv")